### Importing data (csv)

In [40]:
import pandas as pd
import glob

**Reading rental csv files**

In [41]:
csv_files = glob.glob('data_files/data/JC*.csv')

df_list = []
for csv in csv_files:
   df_temp = pd.read_csv(csv)
   df_list.append(df_temp)

rentals = pd.concat(df_list) 

**Reading weather data**

In [42]:
weather = pd.read_csv('data_files/data/newark_airport_2016.csv')

**Viewing the dataframes**

In [43]:
rentals.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.0,2
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.0,1
2,202,2016-01-01 00:18:25,2016-01-01 00:21:47,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24689,Subscriber,1962.0,2
3,248,2016-01-01 00:23:13,2016-01-01 00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,-74.044247,24693,Subscriber,1984.0,1
4,903,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Customer,NaN,0


In [44]:
weather.head()

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,TSUN,WDF2,WDF5,WSF2,WSF5
0,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-01,12.75,NaN,0.0,0.0,0.0,41,43,34,NaN,270,280.0,25.9,35.1
1,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-02,9.40,NaN,0.0,0.0,0.0,36,42,30,NaN,260,260.0,21.0,25.1
2,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-03,10.29,NaN,0.0,0.0,0.0,37,47,28,NaN,270,250.0,23.9,30.0
3,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-04,17.22,NaN,0.0,0.0,0.0,32,35,14,NaN,330,330.0,25.9,33.1
4,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-05,9.84,NaN,0.0,0.0,0.0,19,31,10,NaN,360,350.0,25.1,31.1


In [45]:
rentals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 247584 entries, 0 to 15113
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Trip Duration            247584 non-null  int64  
 1   Start Time               247584 non-null  object 
 2   Stop Time                247584 non-null  object 
 3   Start Station ID         247584 non-null  int64  
 4   Start Station Name       247584 non-null  object 
 5   Start Station Latitude   247584 non-null  float64
 6   Start Station Longitude  247584 non-null  float64
 7   End Station ID           247584 non-null  int64  
 8   End Station Name         247584 non-null  object 
 9   End Station Latitude     247584 non-null  float64
 10  End Station Longitude    247584 non-null  float64
 11  Bike ID                  247584 non-null  int64  
 12  User Type                247204 non-null  object 
 13  Birth Year               228585 non-null  float64
 14  Gender    

### Cleaning the dataframes

In [46]:
maxRow = rentals['Bike ID'].count()

print((1- rentals.count()/maxRow)*100)

Trip Duration              0.000000
Start Time                 0.000000
Stop Time                  0.000000
Start Station ID           0.000000
Start Station Name         0.000000
Start Station Latitude     0.000000
Start Station Longitude    0.000000
End Station ID             0.000000
End Station Name           0.000000
End Station Latitude       0.000000
End Station Longitude      0.000000
Bike ID                    0.000000
User Type                  0.153483
Birth Year                 7.673759
Gender                     0.000000
dtype: float64


**Since the missing data in any of the columns is not greater than 60%, so we cannot delete any row**

In [47]:
# finding the null values
rentals.isnull().sum()

Trip Duration                  0
Start Time                     0
Stop Time                      0
Start Station ID               0
Start Station Name             0
Start Station Latitude         0
Start Station Longitude        0
End Station ID                 0
End Station Name               0
End Station Latitude           0
End Station Longitude          0
Bike ID                        0
User Type                    380
Birth Year                 18999
Gender                         0
dtype: int64

**Using multiple imputation to fill the missing birth years**

In [48]:
# Using multiple imputation to fill the missing birth years

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp = IterativeImputer(max_iter=10, random_state=0)
rentals['Birth Year'] = imp.fit_transform(rentals[['Birth Year']])

In [49]:
rentals['Birth Year'].isnull().sum()

0

**To fill the null values in user type column the best way is to use mode imputation. As the type which occurs the maximum time can be safely used in place of null values**

In [50]:
# using mode()[0] as mode() returns a series and mode value is at index 0
rentals['User Type'].fillna(rentals['User Type'].mode()[0], inplace=True)

In [51]:
rentals['User Type'].isnull().sum()

0

**Since gender column has int values its better to use a string to denote the genders for better understanding**

In [52]:
gender_map = {0:'Unknown',1:'Male', 2:'Female'}
rentals['Gender_Info'] = rentals['Gender'].map(gender_map)

In [53]:
rentals.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,Gender_Info
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.000000,2,Female
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.000000,1,Male
2,202,2016-01-01 00:18:25,2016-01-01 00:21:47,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24689,Subscriber,1962.000000,2,Female
3,248,2016-01-01 00:23:13,2016-01-01 00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,-74.044247,24693,Subscriber,1984.000000,1,Male
4,903,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Customer,1979.335276,0,Unknown


**Converting the time columns into time formats**

In [54]:
rentals['Start Time'] = pd.to_datetime(rentals['Start Time'], format="%Y-%m-%d %H:%M:%S")
rentals['Stop Time'] = pd.to_datetime(rentals['Stop Time'], format="%Y-%m-%d %H:%M:%S")

**Cleaning the weather data**

In [55]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   STATION  366 non-null    object 
 1   NAME     366 non-null    object 
 2   DATE     366 non-null    object 
 3   AWND     366 non-null    float64
 4   PGTM     0 non-null      float64
 5   PRCP     366 non-null    float64
 6   SNOW     366 non-null    float64
 7   SNWD     366 non-null    float64
 8   TAVG     366 non-null    int64  
 9   TMAX     366 non-null    int64  
 10  TMIN     366 non-null    int64  
 11  TSUN     0 non-null      float64
 12  WDF2     366 non-null    int64  
 13  WDF5     364 non-null    float64
 14  WSF2     366 non-null    float64
 15  WSF5     364 non-null    float64
dtypes: float64(9), int64(4), object(3)
memory usage: 45.9+ KB


In [56]:
weather.isnull().sum()

STATION      0
NAME         0
DATE         0
AWND         0
PGTM       366
PRCP         0
SNOW         0
SNWD         0
TAVG         0
TMAX         0
TMIN         0
TSUN       366
WDF2         0
WDF5         2
WSF2         0
WSF5         2
dtype: int64

* TSUN (daily total sunshine can be filled with average values)
* PGTM (Peak Gust Time can also be filled with average values)
* But in our case the entire column has null values

Let's just replace null values with 0

In [57]:
weather.fillna(0, inplace=True)

In [58]:
rentals.nunique()

Trip Duration                6024
Start Time                 244407
Stop Time                  244137
Start Station ID               51
Start Station Name             51
Start Station Latitude         51
Start Station Longitude        51
End Station ID                102
End Station Name              102
End Station Latitude          102
End Station Longitude         102
Bike ID                       566
User Type                       2
Birth Year                     65
Gender                          3
Gender_Info                     3
dtype: int64

In [59]:
weather['NAME'] = weather['NAME'].str.replace(',', '')

In [60]:
weather.NAME

0      NEWARK LIBERTY INTERNATIONAL AIRPORT NJ US
1      NEWARK LIBERTY INTERNATIONAL AIRPORT NJ US
2      NEWARK LIBERTY INTERNATIONAL AIRPORT NJ US
3      NEWARK LIBERTY INTERNATIONAL AIRPORT NJ US
4      NEWARK LIBERTY INTERNATIONAL AIRPORT NJ US
                          ...                    
361    NEWARK LIBERTY INTERNATIONAL AIRPORT NJ US
362    NEWARK LIBERTY INTERNATIONAL AIRPORT NJ US
363    NEWARK LIBERTY INTERNATIONAL AIRPORT NJ US
364    NEWARK LIBERTY INTERNATIONAL AIRPORT NJ US
365    NEWARK LIBERTY INTERNATIONAL AIRPORT NJ US
Name: NAME, Length: 366, dtype: object

#### cleaning the year data

In [61]:
rentals = pd.read_csv('cleaned_datasets/rentals.csv')

In [62]:
rentals['Birth Year'] = round(rentals['Birth Year'], 0)
rentals['Birth Year'] = rentals['Birth Year'].astype(str)

In [63]:
rentals['Birth Year'] = rentals['Birth Year'].str.replace('.0', '')

In [64]:
rentals['Birth Year']

0         1964
1         1962
2         1962
3         1984
4         1979
          ... 
247579    1981
247580    1979
247581    1978
247582    1960
247583    1956
Name: Birth Year, Length: 247584, dtype: object

#### Converting the dataframes back to csv after cleaning

In [65]:
rentals.to_csv('rentals.csv', index=False)
weather.to_csv('weather.csv', index=False)